In [14]:
from flask import Flask, render_template, request, url_for
from tensorflow.keras.models import load_model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model
from tensorflow.keras.utils import img_to_array
from tqdm import tqdm
import keras
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from test_data_in import image_encode
import os
import json
# from requests_toolbelt.multipart import decoder


In [ ]:
# 이미지의 벡터화를 진행하는 코드
# inception v3 model 로드
model_v3 = InceptionV3(weights='imagenet')
# inception v3에서 마지막 레이어를 제거한 새로운 모델 생성
model_new = Model(model_v3.input, model_v3.layers[-2].output)
object_model=load_model('./model/object_model.h5')
scenery_model=load_model('./model/scenery_model.h5')
img_df = pd.read_csv("./csv/시연데이터최종본.csv")

In [17]:
# Function to encode a given image into a vector of size (2048, )
def encode(image_path):
    global model_new
    img = keras.preprocessing.image.load_img(image_path, target_size=(299, 299))
    # Convert image to numpy array of 3-dimensions
    x = img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    fea_vec = model_new.predict(x) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec   # 이 데이터가 input data로 사용됨

In [54]:
def image_encode(img_lst = [], category = "object"):
    global object_model, scenery_model
    image_path='./static/test_img/'    # 분석할 이미지가 있는 디렉토리 지정 필요
    data=[]
    for image in tqdm(img_lst):
        path=image_path+image
        data.append( np.array(encode(path)) )
    data = np.array(data)
    if category == "object":
        # 객체검출 결과
        y_object_pred=[]
        label=object_model.predict(data)
        for sample in label:
            y_object_pred.append([1 if i>=0.5 else 0 for i in sample ] )
        return np.array(y_object_pred)
    elif category == "scenery":
        # 풍경검출 결과
        y_scenery_pred=[]
        label=scenery_model.predict(data)
        for sample in label:
            y_scenery_pred.append([1 if i>=0.5 else 0 for i in sample ] )
        return np.array(y_scenery_pred)

In [36]:
def keyword_img_match(keyword):
    global img_df
    img_lst = []
    for i in img_df.index:
        if keyword in img_df.iloc[i,1]:
            img_lst.append(img_df.iloc[i,0])
    return img_lst

In [72]:
def keyword_search_len(keyword):
    lst = 0
    for i in img_df.index:
        if keyword in img_df.iloc[i,1]:
            lst+=1
    return lst

In [90]:
total_keywords = ['꽃',  '꽃스타그램','먹스타그램', '맛스타그램', '먹방', '맛집', '술스타그램', '디저트', '아침', '집밥', '먹스타', '맛있다',
                    '냠냠', '야식', '맛있다그램', '먹부림', '요리', '마카롱', '간식', '부산맛집', '치킨', '온더테이블', '존맛', '푸드스타그램', 
                    '음식', '제주맛집', '점심', '저녁','데일리룩', '오오티디', '패션', '옷스타그램', '패션스타그램', '뷰티', '스타일',
                    '모델', '코디', '명품',  '뷰티스타그램', '아웃핏', '데일리코디', '등원룩', '원피스', '옷', '메이크업', '네일아트', '젤네일',
                    '네일', '네일스타그램', '럽스타그램', '데이트', '커플', '사랑', '내사랑', '육아', '육아스타그램', '육아소통', '육아맘',
                    '맘스타그램', '아들스타그램', '애스타그램', '도치맘', '인스타베이비', '아들맘', '아들', '딸바보', '가족', '딸', '육아일기',
                    '딸맘', '베이비스타그램', '아기', '육아일상', '아기스타그램', '부부스타그램', '줌마그램', '아들바보', '베이비', '가족여행',
                    '베이비그램', '육아그램', '신혼', '신혼부부', '닭띠맘', '원숭이띠아기', '가족스타그램', '닭띠아기', '딸스타그램', '고양이',
                    '반려견', '강아지', '냥스타그램', '개스타그램', '펫스타그램', '댕댕이', '캣스타그램', '독스타그램', '견스타그램', '푸들',
                    '말티즈', '반려묘', '멍멍이', '포메라니안', '반려동물']
key_counts = {}
for words in total_keywords:
    counts = 0
    for tags in img_df["tags"].values:
        if words in tags:
            counts += 1
    key_counts[words] = str(counts)

In [95]:
app = Flask(__name__)

filename = ""
img_lst = []

# 웹 페이지 접속시 메인 화면
@app.route('/',  methods=['GET'])
def main_loading():
    return render_template("index.html")

@app.route("/search", methods=["GET"])
def search_page():
    global img_lst
    keyword = request.args.get("keyword")
    img_lst = keyword_img_match(keyword)
    labeld_lst = image_encode(img_lst)

    if len(img_lst) < 12:
        return render_template("search.html",img_names = img_lst, keyword = keyword, img_len = len(img_lst))
    else:
        return render_template("search.html",img_names = img_lst[:12], keyword = keyword, img_len = len(img_lst))

@app.route("/keywords", methods = ["POST"])
def keywords_search():
    global total_keywords, key_counts
    keyword = request.form["keyword"]
    
    keywords = {}
    cnt = 0
    # 검색어가 있는 키워드 필터링
    for word in total_keywords:
        if keyword in word:
                                  # 키워드  ,    갯수
            keywords[str(cnt)] = [word, key_counts[word]]
            cnt+=1
    print(keyword)
    print(keywords)
    return json.dumps(keywords)

@app.route("/image_append", methods=["POST"])
def image_append():
    global img_lst
    keyword = request.get_json()["keyword"]
    num = int(request.get_json()["num"])
    img_dict = [{names:names} for names in img_lst]
    if num + 13 >= len(img_dict):
        return json.dumps(img_dict[num:])
    else:
        return json.dumps(img_dict[num:num+12])

  
if __name__ == "__main__":

    app.run()  

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
100%|██████████| 9/9 [00:01<00:00,  5.60it/s]
127.0.0.1 - - [23/Jul/2022 00:16:01] "GET /search?keyword=아웃핏 HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2022 00:16:01] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:01] "GET /static/img/insta_logo.png HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:01] "GET /static/nav.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:01] "GET /static/search_script.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:01] "GET /static/img/compass_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:01] "GET /static/img/cancle_button.png HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:01] "GET /static/img/home_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:01] "GET /static/img/paper_plane.png HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:01] "GET /static/test_img/9010559.jpg HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:01] "GET /static/img/plus_icon.png 

아웃핏
{'0': ['아웃핏', '9']}
아웃
{'0': ['아웃핏', '9']}
아웃
{'0': ['아웃핏', '9']}
아
{'0': ['아침', '29'], '1': ['아웃핏', '9'], '2': ['네일아트', '44'], '3': ['육아', '209'], '4': ['육아스타그램', '81'], '5': ['육아소통', '99'], '6': ['육아맘', '77'], '7': ['아들스타그램', '51'], '8': ['아들맘', '52'], '9': ['아들', '133'], '10': ['육아일기', '38'], '11': ['아기', '145'], '12': ['육아일상', '40'], '13': ['아기스타그램', '35'], '14': ['아들바보', '29'], '15': ['육아그램', '22'], '16': ['원숭이띠아기', '11'], '17': ['닭띠아기', '10'], '18': ['강아지', '61']}


127.0.0.1 - - [23/Jul/2022 00:16:19] "POST /keywords HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2022 00:16:19] "POST /keywords HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2022 00:16:19] "POST /keywords HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2022 00:16:19] "POST /keywords HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2022 00:16:19] "POST /keywords HTTP/1.1" 200 -


아
{'0': ['아침', '29'], '1': ['아웃핏', '9'], '2': ['네일아트', '44'], '3': ['육아', '209'], '4': ['육아스타그램', '81'], '5': ['육아소통', '99'], '6': ['육아맘', '77'], '7': ['아들스타그램', '51'], '8': ['아들맘', '52'], '9': ['아들', '133'], '10': ['육아일기', '38'], '11': ['아기', '145'], '12': ['육아일상', '40'], '13': ['아기스타그램', '35'], '14': ['아들바보', '29'], '15': ['육아그램', '22'], '16': ['원숭이띠아기', '11'], '17': ['닭띠아기', '10'], '18': ['강아지', '61']}
아ㅊ
{}
아ㅊ
{}
아치
{}
아치
{}


127.0.0.1 - - [23/Jul/2022 00:16:19] "POST /keywords HTTP/1.1" 200 -


아침
{'0': ['아침', '29']}


127.0.0.1 - - [23/Jul/2022 00:16:20] "GET /static/img/glass.png HTTP/1.1" 304 -
100%|██████████| 29/29 [00:04<00:00,  5.95it/s]
127.0.0.1 - - [23/Jul/2022 00:16:25] "GET /search?keyword=아침 HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2022 00:16:25] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:25] "GET /static/img/insta_logo.png HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:25] "GET /static/img/home_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:25] "GET /static/img/cancle_button.png HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:25] "GET /static/nav.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:25] "GET /static/test_img/9002603.jpg HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:25] "GET /static/img/plus_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:25] "GET /static/search_script.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:25] "GET /static/test_img/9003572.jpg HTTP/1.1" 304 -
127.0.0.1 - - [23/Jul/2022 00:16:25] "GET /stati